In [ ]:
## Uniprot_crawling
import json
import openpyxl
import requests  # 导入requests包
from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning
import pandas as pd

wb = openpyxl.Workbook()
sheet1 = wb.active
sheet1.append(['uniport', 'Name', 'Sequence', 'family', 'PDBID', 'AlphaFoldDB', 'taxonId', 'organism', 'function', 'fun_pub', 'mfun', 'pmid'])
sheet2 = wb.create_sheet('title')
sheet2.title = 'new sheet'
sheet1.sheet_properties.tabColor = '000000'
element = []
i, j = 0, 0
disable_warnings(InsecureRequestWarning)
if __name__ == '__main__':
    headers = {
        'User-Agent': "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/103.0.0.0 Mobile Safari/537.36 "
    }

    url_main0 = 'https://alphafold.ebi.ac.uk/api/prediction/'
    url_main1 = 'https://rest.uniprot.org/uniprotkb/'  # 名字
    # keyword = pd.read_excel('fileName.xlsx', dtype=object).loc[:, 'Column_Name']
    keyword = [ 'P01023', 'P58455', 'P02778', 'P05106', 'P84640', 'P07477', 'P07550', 'P07822', 'P08069'] #test data
    for i in range(len(keyword)):
        url1 = url_main1 + str(keyword[i]) + '/publications'
        response1 = requests.get(url=url1, headers=headers, verify=False)
        response1.encoding = response1.apparent_encoding
        data1 = json.loads(response1.text)
        url = url_main1 + str(keyword[i])
        response = requests.get(url=url, headers=headers, verify=False)
        response.encoding = response.apparent_encoding
        data = json.loads(response.text)
        refer_list = data1.get('results')
        Name, Sequence, family ,function, fun_evidence, PDBID, AlphaFoldDB, taxonId = '', '', '', '', '', '', '', ''
        print('https://www.uniprot.org/uniprotkb/'+ str(keyword[i]))
        Name = data.get('uniProtkbId')
        Sequence = data.get('sequence').get('value')
        try:
            if data.get('organism').get('commonName'):
                organism = data.get('organism').get('commonName')
            else:
                organism = data.get('organism').get('scientificName')
            taxonId = data.get('organism').get('taxonId')
        except:
            organism = ''
            taxonId = ''
        References_list = data.get('uniProtKBCrossReferences')
        for ref in range(len(References_list)):
            PDBID = References_list[ref].get('id') if References_list[ref].get('database') == 'PDB' else PDBID
            AlphaFoldDB =  References_list[ref].get('id') if References_list[ref].get('database') == 'AlphaFoldDB' else AlphaFoldDB   
              
        if data.get('comments'):
            comments_list = data.get('comments')
            for j in range(len(comments_list)):
                if comments_list[j].get('commentType') == 'SIMILARITY':
                    family = ''.join(comments_list[j].get('texts')[0].get('value'))
        pmid_list = data1.get('results')
        try:
            for pm in range(len(pmid_list)):
                database = pmid_list[pm].get('citation').get('citationCrossReferences')[0].get('database')
                if database == 'PubMed':
                    pmid = pmid_list[pm].get('citation').get('citationCrossReferences')[0].get('id')
        except:
            pmid = ''
        if data.get('comments'):
            if data.get('comments')[0].get('commentType') == 'FUNCTION':
                function = ''.join(data.get('comments')[0].get('texts')[0].get('value'))
                fun_evidence_list = data.get('comments')[0].get('texts')[0].get('evidences')
                try:
                    for fun in range(len(fun_evidence_list)):
                        fun_evidence = ''.join(fun_evidence_list[fun].get('id')) + ';' + fun_evidence
                except:
                    fun_evidence = fun_evidence
        mfun_list = data.get('keywords')
        mfun = ''
        if mfun_list:
            for m in range(len(mfun_list)):
                mfun = ''.join(mfun_list[m].get('name')) + ';' + mfun if mfun_list[m].get('category') == 'Molecular function' else mfun
                    
        xx_info = [keyword[i], Name, Sequence, family, organism, PDBID, AlphaFoldDB, taxonId, function, fun_evidence, mfun, pmid]
        sheet1.append(xx_info)
    wb.save('uniport_organism.xlsx')